# FROM SCRIPT TO SCREEN: WHAT MAKES BOX OFFICE HIT?

*Project by: GROUP 8*

1.Deborah Okeyo

2.Marion Macharia

3.David Chege

4.Evans Oyugi

# 1. Introduction

# 1.1 Overview

Applying exploratory data analysis to provide insights for a Microsoft stakeholder on which types of films to produce.

## 1.2 Business Understanding

*Objective*

The goal of this project is to use exploratory data analysis (EDA) to draw insights to aid the making of a new movie studio looking to enter the film industry and needs to determine which types of films are currently est performing at the box office. By analyzing various datasets related to movie performance. The objective is to identify trends and characteristics of successful films that can guide the studio’s content creation strategy.

*Business context*

With the evolution of film industry, understanding the market and consumer preferences is essential for new entrants. A new movie studio must leverage data-driven insights to make informed decisions about film production. The business wants to identify which factors contribute to box office success to maximize their investment and ensure their film offerings align with market demands.

# 2. Data Understanding

## 2.1 Data

The folder containing the data is called "zippedData" which contains movie datasets from:

1. Box Office Mojo- an online database providing detailed information on box office revenues

2. IMDB- a comprehensive database of movie and TV information

3. Rotten Tomatoes- a review aggregation website for films and TV shows

4. TheMovieDB- a community-driven movie and TV database

5. The Numbers- a database providing box office revenue information and other film metrics

The files are in different formats namely CSV and TSV files and can be opened using the pd.read_csv.
Data from IMDB is in a SQLite Database.

Data from the TSV files was not used because it didn't have the data neede for this analysis. Only the compressed CSV files and unzipped IMDB file were used.

Dataset used has 1001 rows and 10 columns. The columns are:

1. movie - name of film

2. runtime_minutes - total minutes of a film

3. genres - types of films

4. averagerating - average rating of individual films

5. production_budget - total production budget for the film

6. month - release month 

7. profit - total amount remaining production budget from total gross

8. original_language - film language

9. popularity - how popular a film is

10. year - release year 

In order to provide recommendations, this analysis will address the following questions:

1.What genres of films are performing best at the box office?

2.Which languages are most successful in terms of box office revenue?

3.How does the month of release impact profitability?

4.Is there a correlation between the duration of films and other factors?

## 2.2 Loading Libraries

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sqlite3
import shutil
import warnings
warnings.filterwarnings("ignore")
from collections import Counter

## 2.3 Loading the Data

In [3]:
#Creating a connection
conn = sqlite3.connect("im.db")
cur = conn.cursor()

In [4]:
# Fetching table names
cur.execute(""" SELECT name FROM sqlite_master WHERE type='table';""")
tables = cur.fetchall()

In [5]:
# Loading csv datasets
budget_df = pd.read_csv("tn.movie_budgets.csv")
votes_df = pd.read_csv("tmdb.movies.csv", index_col=0)
gross_df = pd.read_csv("bom.movie_gross.csv")

#loading the sql tables datasets 
basics_df = pd.read_sql("""SELECT * FROM movie_basics;""", conn)
ratings_df = pd.read_sql("""SELECT * FROM movie_ratings;""", conn)

## 2.4 Exploring the data

In [6]:
# the first 2 values in the budget dataset
budget_df.head(5)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [7]:
# Information about the dataset i.e., number of rows and columns, column names, missing values and datatype
budget_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [8]:
votes_df.head(5)

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [9]:
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [10]:
gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [11]:
basics_df.shape

(146144, 6)

In [12]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


## Budget Dataset

In [13]:
# Datatypes of the dataset should be correct but first we need to know them
budget_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [14]:
# Format the date into the pandas date format
budget_df['release_date'] = pd.to_datetime(budget_df['release_date'])

In [15]:
# We want to operate on multiple columns hence put them in a list:
budget_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']

# pass them to df.replace(), specifying each char and it's replacement:
budget_df[budget_cols] = budget_df[budget_cols].replace({'\$': '', ',': ''}, regex=True)

In [16]:
# Convert the budget and gross columns into the integer datatype
budget_df[budget_cols] = budget_df[budget_cols].astype(np.int64)

In [17]:
# Checking to see if the values for the columns have been converted to the necessary datatypes
budget_df.head(5)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [18]:
# Checking for duplicates in the dataset
budget_df.duplicated().sum()

0

In [19]:
budget_df.duplicated(["movie","release_date"]).sum()

0

In [20]:
# Checking for missing values
budget_df.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

## Votes dataset

In [21]:
# We will look at the votes dataset as we did above with the budget dataset
votes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB


In [22]:
# Format the date into the pandas date format
votes_df['release_date'] = pd.to_datetime(votes_df['release_date'])

In [23]:
# Checking for duplicates in the dataset
votes_df.duplicated().sum()

1020

In [24]:
# Remove the duplicates
votes_df.drop_duplicates(keep="first", inplace=True)

In [25]:
# Confirming that the duplicates have been removed
votes_df.duplicated().sum()

0

In [26]:
#Checking for duplicates in the title column
votes_df.duplicated(["title","release_date"]).sum()

7

In [27]:
votes_df.drop_duplicates(["title","release_date"], keep="first", inplace=True)

In [28]:
votes_df.duplicated(["title","release_date"]).sum()

0

In [29]:
# Checking for missing values
votes_df.isna().sum()

genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

## Gross dataset

In [30]:
gross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [31]:
gross_df.head(5)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [32]:
# removing the quotations and commas in the foreign gross column
gross_df["foreign_gross"] = gross_df["foreign_gross"].replace({"''": "", ",": ""}, regex=True)

In [33]:
# Changing the datatype from object to float
gross_df["foreign_gross"] = gross_df["foreign_gross"].astype(float)

In [34]:
# Checking for duplicates
gross_df.duplicated().sum()

0

In [35]:
# Check for duplicates in the title column
gross_df.duplicated(["title","year"]).sum()

0

In [36]:
# Check for missing values 
gross_df.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

# Basics Dataset

In [37]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [38]:
basics_df.head(5)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [39]:
#Check for duplicates
basics_df.duplicated().sum()

0

In [40]:
# Check for duplicates in the primary title column
basics_df.duplicated(["primary_title","start_year"]).sum()

2072

In [41]:
# drop the duplicates in the primary title column
basics_df.drop_duplicates(["primary_title","start_year"], keep="first", inplace=True)

In [42]:
# Check for duplicates in the primary title column
basics_df.duplicated(["primary_title","start_year"]).sum()

0

In [43]:
# Check for missing values 
basics_df.isna().sum()

movie_id               0
primary_title          0
original_title        18
start_year             0
runtime_minutes    31218
genres              5192
dtype: int64

In [44]:
# We will drop the original title column because there is a primary title column
basics_df = basics_df.drop('original_title', axis = 1)

In [45]:
# We will drop the rows with missing values in the genres column
basics_df = basics_df.dropna(subset=['genres'])

In [46]:
# Drop the rows with missing values in the runtime minutes column
basics_df = basics_df.dropna(subset=['runtime_minutes'])

In [47]:
# Check for any more missing values
basics_df.isna().sum()

movie_id           0
primary_title      0
start_year         0
runtime_minutes    0
genres             0
dtype: int64

# Ratings Dataset